In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestRegressor
import bz2
import _pickle as cPickle

In [2]:
df = pd.read_csv("agridata_csv_202110311352.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 836977 entries, 0 to 836976
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   commodity_name  836977 non-null  object 
 1   state           836977 non-null  object 
 2   district        836977 non-null  object 
 3   market          836977 non-null  object 
 4   min_price       836976 non-null  float64
 5   max_price       836976 non-null  float64
 6   modal_price     836976 non-null  float64
 7   date            836976 non-null  object 
dtypes: float64(3), object(5)
memory usage: 51.1+ MB


In [4]:
df.head()

,commodity_name,state,district,market,min_price,max_price,modal_price,date
0,Ajwan,Gujarat,Amreli,Amreli,63.25,79.55,79.55,22-05-19
1,Ajwan,Gujarat,Banaskanth,Dhanera,80.05,80.05,80.05,22-05-19
2,Ajwan,Gujarat,Jamnagar,Jamnagar,90.00,155.00,122.50,22-05-19
3,Ajwan,Gujarat,Mehsana,Kadi,94.50,112.50,107.00,22-05-19
4,Ajwan,Gujarat,Mehsana,Unjha,21.25,148.50,99.75,22-05-19


In [5]:
df = df.dropna()

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 836976 entries, 0 to 836976
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   commodity_name  836976 non-null  object 
 1   state           836976 non-null  object 
 2   district        836976 non-null  object 
 3   market          836976 non-null  object 
 4   min_price       836976 non-null  float64
 5   max_price       836976 non-null  float64
 6   modal_price     836976 non-null  float64
 7   date            836976 non-null  object 
dtypes: float64(3), object(5)
memory usage: 57.5+ MB


In [7]:
X = df.iloc[:,:-2]

In [8]:
rf = RandomForestRegressor()

In [9]:
le  = LabelEncoder()

In [10]:
X['commodity_name'] = le.fit_transform(X['commodity_name'])
X['state'] = le.fit_transform(X['state'])
X['district'] = le.fit_transform(X['district'])
X['market'] = le.fit_transform(X['market'])

In [11]:
Y = df['modal_price']

In [12]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.3,random_state=42)

In [13]:
rf.fit(X_train,Y_train)

RandomForestRegressor()

In [14]:
rf.score(X_test,Y_test)

0.9711153468461666

In [15]:
le_c = LabelEncoder()
le_s = LabelEncoder()
le_d = LabelEncoder()
le_m = LabelEncoder()

In [17]:
deploy = {'model':rf , 'le_c': le_c,'le_s':le_s,'le_d':le_d,'le_m':le_m}
file = bz2.BZ2File('saved_steps.pkl.bz2','wb')
cPickle.dump(deploy,file)
file.close()

In [17]:
file = bz2.BZ2File('saved_steps.pkl.bz2','rb')
mod = pickle.load(file)

In [18]:
df['market'].unique()

array(['Amreli', 'Dhanera', 'Jamnagar', ..., 'Hingoli(Kanegoan Naka)',
       'Jalgaon Jamod(Aasalgaon)', 'Nagarkurnool'], dtype=object)